***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [2. Mathematical Groundwork](2_0_introduction.ipynb)
    * Previous: [2.7 Fourier Theorems](2_7_fourier_theorems.ipynb)
    * Next: [2.9 Sampling Theory](2_9_sampling_theory.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

Import section specific modules:

In [ ]:
pass

1. [The Discrete Fourier transform](#)
    1. [The Discrete Fourier transform: definition](#math:sec:the_discrete_fourier_transform_definition)
    2. [The Discrete convolution: definition and discrete convolution theorem](#math:sec:the_discrete_convolution_definition_and_discrete_convolution_theorem)
    3. [The Discrete Fourier transform as coefficients for the Fourier transform of a sampled function](#math:sec:the_discrete_fourier_transform_as_coefficients_for)
    4. [Fast Fourier transforms](#math:sec:fast_fourier_tranforms)


## 2.8. The Discrete Fourier Transform (DFT) and the Fast Fourier Transform (FFT)<a id='math:sec:the_discrete_fourier_transform_and_the_fast_fourier_transform'></a>

In the context of many applications (always in an experiment), the functional description of a phenomenon is a discrete function (or a series) with finite support (in other words, very often we describe a phenomenon with a finite set of numbers). Also here, the Fourier transform is an extremely important tool. Computationally, in fact, a Fourier transform can only be implemented as the discrete equivalent of a Fourier transform, not only disrete, but also with a limited amount of numbers. In this chapter, we see how the concept of a Fourier transform and the connected identities extend to a finite set of numbers.

### 2.8.1. The Discrete Fourier Transform: Definition<a id='math:sec:the_discrete_fourier_transform_definition'></a>

Let $x = \left\{x_n \in \mathbb{C}\right\}_{n = 1, \ldots, N}$ a finite set of complex numbers. Then, the discrete Fourier transform $\mathscr{F}_{\rm D}$ is defined as

<a id='math:eq:8_001'></a><!--\label{math:eq:8_001}-->$$
\mathscr{F}_{\rm D}: \left\{y_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1} \rightarrow \left\{Y_k \in \mathbb{C}\right\}_{k \in \mathbb{Z}}\\
\begin{align}
y \,&=\, \left\{y_n \in \mathbb{C}\right\}_{n = 1, \ldots, N}\\
&\Rightarrow\\
\mathscr{F}_{\rm D}\{y\} &=\, \left\{Y_k\in\mathbb{C}\right\}_{k \in \mathbb{Z}}\\
Y_k\,&=\,\sum_{n\,=\,0}^{N-1} y_n\,e^{-\imath 2\pi \frac{nk}{N}}
\end{align}\qquad {\rm .}
$$

The inverse discrete Fourier transform is defined as

<a id='math:eq:8_002'></a><!--\label{math:eq:8_002}-->$$
\mathscr{F}_{\rm D}^{-1}: \left\{Y_k \in \mathbb{C}\right\}_{k \,=\, 0, \ldots, N-1} \rightarrow \left\{y_n \in \mathbb{C}\right\}_{n \in \mathbb{Z}}\\
\begin{align}
Y \,&=\, \left\{y_k \in \mathbb{C}\right\}_{k = 1, \ldots, N}\\
&\Rightarrow\\
\mathscr{F}_{\rm D}^{-1}\{Y\} &=\, \frac{1}{N}\,\left\{Y_n\in\mathbb{C}\right\}_{n \in \mathbb{Z}}\\
y_n\,&=\,\sum_{k\,=\,0}^{N-1} Y_k\,e^{\imath 2\pi \frac{nk}{N}}
\end{align}\qquad {\rm .}
$$

Depending on its implementation, the normalisation factor $\frac{1}{N}$ sometimes appears in front of the forward or the backward transformation, is sometimes shared by putting a $\sqrt{\frac{1}{N}}$ in front of the sum, or is sometimes omitted completely. The inverse discrete Fourier transform is the inverse operation with respect to the discrete Fourier transform (restricted to the original domain):

<a id='math:eq:8_003'></a><!--\label{math:eq:8_003}-->$$
\begin{align}
\mathscr{F}_{\rm D}^{-1}\left\{\mathscr{F}_{\rm D}\left\{y\right\}\right\}_{n^\prime} \,&=\, \frac{1}{N}\sum_{k\,=\,0}^{N-1} \left(\sum_{n\,=\,0}^{N-1} y_n e^{-\imath 2\pi\frac{kn}{N}}\right)e^{\imath 2\pi\frac{kn^\prime}{N}}\\
&=\,\frac{1}{N}\sum_{k\,=\,0}^{N-1} \sum_{n\,=\,0}^{N-1} \left( y_n e^{-\imath 2\pi\frac{kn}{N}}e^{\imath 2\pi\frac{kn^\prime}{N}}\right)\\
&=\,\frac{1}{N}\left(\sum_{k\,=\,0}^{N-1} y_{n^\prime}+\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} \sum_{k\,=\,0}^{N-1} y_n e^{-\imath 2\pi\frac{kn}{N}}e^{\imath 2\pi\frac{kn^\prime}{N}}\right)\\
&=\,\frac{1}{N}\left(\sum_{k\,=\,0}^{N-1} y_{n^\prime}+\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} \sum_{k\,=\,0}^{N-1} y_n e^{\imath 2\pi\frac{k(n^\prime-n)}{N}}\right)\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \sum_{k\,=\,0}^{N-1} \left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)^k\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \frac{1-\left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)^N}{1-\left(e^{\imath 2\pi\frac{(n^\prime-n)}{N}}\right)}\\
&=\,y_{n^\prime}+\frac{1}{N}\sum_{\begin{split}n\,&=\,0\\n\,&\neq\,n^\prime\end{split}}^{N-1} y_n \frac{1-e^{\imath 2\pi(n^\prime-n)}}{1-e^{\imath 2\pi\frac{(n^\prime-n)}{N}}}\\
&\underset{n,n^\prime \in \mathbb{N}}{=}\,y_{n^\prime}\\
\end{align}\qquad {rm ,}
$$

where we made use of the identity $\sum_{n\,=\,0}^{N-1}x^n \,=\, \frac{1-x^N}{1-x}$. Also for the discrete Fourier transform, it is possible to express the inverse transform in terms of the forward transform (without proof, but it's straightforward):

<a id='math:eq:8_004'></a><!--\label{math:eq:8_004}-->$$
\begin{align}
\mathscr{F}_{\rm D}^{-1}\{Y\}_n \,&=\, \frac{1}{N} \mathscr{F}_{\rm D}\{Y\}_{-n} \\
&=\,\frac{1}{N} \mathscr{F}_{\rm D}\{Y\}_{N-n}\\
\end{align}\qquad {\rm ,}
$$

the discrete (inverse) Fourier transform of a real-valued set of numbers is Hermetian (and vice versa)

<a id='math:eq:8_005'></a><!--\label{math:eq:8_005}-->$$
\begin{align}
y \,&=\, \left\{y_n \in \mathbb{R}\right\}_{n\,=\,1, \ldots, \,N\,\in\, \mathbb{N}}\\
&\Rightarrow
\end{align}\\
\begin{split}
\mathscr{F}_{\rm D}\{y\}_k\,&=\, \left(\mathscr{F}_{\rm D}\{y\}\right)^*_{-k}\\
&=\, \left(\mathscr{F}_{\rm D}\{y\}\right)^*_{N-k}
\end{split}
\qquad {\rm .}
$$

### 2.8.2. The Discrete Convolution: Definition and Discrete Convolution Theorem<a id='math:sec:the_discrete_convolution_definition_and_discrete_convolution_theorem'></a>

In analogy to the analytic convolution we define the discrete convolution:

<a id='math:eq:8_006'></a><!--\label{math:eq:8_006}-->$$
\circ: \left\{y_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1}\times \left\{z_n \in \mathbb{C}\right\}_{n \,=\, 0, \ldots, N-1} \rightarrow \left\{r_k \in \mathbb{C}\right\}_{k \in \mathbb{Z}}\\
\begin{align}
y \,&=\, \left\{y_n \in \mathbb{C}\right\}_{n = 1, \ldots, N}\\
z \,&=\, \left\{z_n \in \mathbb{C}\right\}_{n = 1, \ldots, N}\\
&\Rightarrow\\
(y\circ z)_k &=\, \sum_{n\,=\,0}^{N-1} y_n z_{(k-n)\,mod\,N}\\
\end{align}\qquad {\rm ,}
$$

where $mod$ denotes the modulo operation. One can again show (not here) that the discrete convolution is commutative. One important effect evident from this equation is that if the two series are "broad" enough, the convolution will be continued at the beginning of the series, an effect called aliasing.

Also for the discrete Fourier transform and the discrete convolution the convolution theorem is valid (without proof here).

Let $y\cdot z \underset{def}{=} \left\{y_n\cdot z_n\right\}_{n\,=\,1, \ldots, \,N}$. Then

<a id='math:eq:8_005'></a><!--\label{math:eq:8_005}-->$$
\forall N\,\in\, \mathbb{N}\quad,\\
\begin{align}
y \,&=\, \left\{y_n \in \mathbb{C}\right\}_{n\,=\,1, \ldots, \,N}\\
z \,&=\, \left\{z_n \in \mathbb{C}\right\}_{n\,=\,1, \ldots, \,N}\\
Y \,&=\, \left\{y_k \in \mathbb{C}\right\}_{k\,=\,1, \ldots, \,N}\\
Y \,&=\, \left\{z_k \in \mathbb{C}\right\}_{k\,=\,1, \ldots, \,N}\\
\end{align}\\
\begin{split}
\mathscr{F}_{\rm D}\{y\cdot z\}\,&=\,\frac{1}{N}\mathscr{F}_{\rm D}\{y\}\circ \mathscr{F}_{\rm D}\{z\}\\
\mathscr{F}_{\rm D}^{-1}\{Y\cdot Z\}\,&=\,\mathscr{F}_{\rm D}\{y\}\circ \mathscr{F}_{\rm D}\{z\}\\
\mathscr{F}_{\rm D}\{y\circ z\}\,&=\,\mathscr{F}_{\rm D}\{y\}\circ \mathscr{F}_{\rm D}\{z\}\\
\mathscr{F}_{\rm D}^{-1}\{Y\circ Z\}\,&=\,\frac{1}{N}\mathscr{F}_{\rm D}\{y\}\circ \mathscr{F}_{\rm D}\{z\}\\
\end{split}
\qquad {\rm .}
$$

### 2.8.3. The Discrete Fourier Transform as Coefficients for the Fourier Transform of a Sampled Function<a id='math:sec:the_discrete_fourier_transform_as_coefficients_for_the_fourier_transform_of_a_sampled_function'</a>

Until now we have shown that the discrete Fourier transform shares many properties of the Fourier transform. Here, we discuss how both operations are connected.

For a (Lesbeque-integrable) function $f: \mathbb{R} \rightarrow \mathbb{C}$ define a finitely sampled function $f_{\rm S}$, sampled by the sampling function $s_{x_0, \Delta x, N}$

<a id='math:eq:8_006'></a><!--\label{math:eq:8_006}-->$$
f: \mathbb{R} \rightarrow \mathbb{C}\\
s_{x_0, \,\Delta x,\, N}: \mathbb{R} \rightarrow \mathbb{C}\\
\begin{align}
s_{x_0, \Delta x, N}(x)\,&=\,\frac{1}{\Delta x}III\left(\frac{x-x_0}{\Delta x}\right)\cdot \Pi\left(\frac{x-x_0+\frac{(N-1)\Delta x}{2}}{(N-1)\Delta x}\right)\\
&=\,\sum_{n\,=\,-\infty}^{+\infty}\delta(x-n\Delta x-x_0)\,\Pi\left(\frac{x-x_0+\frac{(N-1)\Delta x}{2}}{(N-1)\Delta x}\right)\\
&=\,\sum_{n\,=\,0}^{N-1}\delta(x-n\Delta x-x_0)\\
&\Rightarrow\\
f_{\rm s}(x)\,&=\,f\cdot s_{x_0, \,\Delta x,\, N}(x)\\
\,&=\,\sum_{n\,=\,0}^{N-1}\delta(x-n\Delta x-x_0)\,f(x)
\end{align}\qquad .
$$

This is nothing else than simulating an experiment in which a function (with potentially infinite support) is measured in $N$ regular spaced steps over a finite range $\left[x_0-\frac{N\Delta x}{2},x_0+\frac{N\Delta x}{2}\right]$. Its Fourier transform is

<a id='math:eq:8_007'></a><!--\label{math:eq:8_007}-->$$
\mathscr{F}\left\{f_{\rm s}\right\}(s)\,=\,\sum_{n\,=\,0}^{N-1}f(x_0+n\Delta x)\,e^{-2\pi \imath(x_0+n\Delta x)s}
\qquad .
$$

Now define the set $y$

<a id='math:eq:8_008'></a><!--\label{math:eq:8_008}-->$$
\begin{align}
y\,&:=\,\left\{y_n\in\mathbb{C}\right\}_{n\,=\, 0,\,\ldots, \,N-1}\\
&=\,\left\{f(x_0+n\Delta x)\right\}_{n\,=\, 0,\,\ldots, \,N-1}\\
\end{align}
\qquad .
$$

Its discrete Fourier transform is defined via

<a id='math:eq:8_009'></a><!--\label{math:eq:8_009}-->$$
\begin{align}
\mathscr{F}_{\rm D}\{y\}_k\,&=\,\sum_{n\,=\,0}^{N-1} y_n\,e^{-\imath 2\pi \frac{nk}{N}}\\
&=\,\sum_{n\,=\,0}^{N-1}f(x_0+n\Delta x)\,e^{-\imath 2\pi \frac{nk}{N}}\\
\end{align}
\qquad ,
$$

and by defining

<a id='math:eq:8_010'></a><!--\label{math:eq:8_010}-->$$
\begin{align}
s_k \,&=\, \frac{k}{N\Delta x} \\
\end{align}
$$

it becomes evident that

<a id='math:eq:8_011'></a><!--\label{math:eq:8_011}-->$$
\begin{align}
 \mathscr{F}\left\{f_{\rm s}\right\}(s_k)\,&=\,\mathscr{F}\left\{f_{\rm s}\right\}\left(\frac{k}{N\Delta x}\right)\\
 &=\,\mathscr{F}_{\rm D}\{y\}_k\,e^{-2\pi\imath x_0 s_k}\\
 &=\,\mathscr{F}_{\rm D}\{y\}_k\,e^{-2\pi\imath \frac{k x_0}{N\Delta x}}\\
\end{align}\qquad ,
$$

in other words the discrete Fourier transform returns the Fourier transform of a sampled function, sampled at points the distance of which is the inverse of the total width over which the function is sampled. What is lost is a phase factor stemming from the position of the bracket within which the function is sampled. A similar relation holds for the inverse discrete Fourier transform and the inverse Fourier transform.

<a id='math:eq:8_012'></a><!--\label{math:eq:8_012}-->$$
F: \mathbb{R} \rightarrow \mathbb{C}\\
S_{s_0, \,\Delta s,\, N}: \mathbb{R} \rightarrow \mathbb{C}\\
\begin{align}
S_{s_0, \Delta x, N}(x)\,&=\,\frac{1}{\Delta x}III\left(\frac{s-s_0}{\Delta x}\right)\cdot \Pi\left(\frac{s-s_0+\frac{(N-1)\Delta s}{2}}{(N-1)\Delta s}\right)\\
&=\,\sum_{n\,=\,-\infty}^{+\infty}\delta(s-n\Delta s-s_0)\,\Pi\left(\frac{s-s_0+\frac{(N-1)\Delta x}{2}}{(N-1)\Delta s}\right)\\
&=\,\sum_{n\,=\,0}^{N-1}\delta(s-n\Delta s-s_0)\\
&\Rightarrow\\
F_{\rm S}(s)\,&=\,f\cdot S_{s_0, \,\Delta s,\, N}(s)\\
\,&=\,\sum_{n\,=\,0}^{N-1}\delta(s-n\Delta s-s_0)\,F(s)
\end{align}
$$

<a id='math:eq:8_008'></a><!--\label{math:eq:8_008}-->$$
\begin{align}
Y\,&:=\,\left\{Y_k\in\mathbb{C}\right\}_{k\,=\, 0,\,\ldots, \,N-1}\\
&=\,\left\{F(s_0+k\Delta s)\right\}_{k\,=\, 0,\,\ldots, \,N-1}\\
\end{align}
$$

<a id='math:eq:8_010'></a><!--\label{math:eq:8_010}-->$$
\begin{align}
x_n \,&=\, \frac{n}{N\Delta s} \\
\end{align}
$$

<a id='math:eq:8_013'></a><!--\label{math:eq:8_013}-->$$
\begin{align}
&\Rightarrow\\
 \mathscr{F}^{-1}\left\{F_{\rm S}\right\}(x_n)\,&=\,\mathscr{F}^{-1}\left\{F_{\rm s}\right\}\left(\frac{n}{N\Delta s}\right)\\
 &=\,\mathscr{F}^{-1}_{\rm D}\{Y\}_n\,e^{-2\pi\imath s_0 x_n}\\
 &=\,\mathscr{F}^{-1}_{\rm D}\{Y\}_n\,e^{-2\pi\imath \frac{n s_0}{N\Delta s}}\\
\end{align}\qquad .
$$

### 2.8.4. Fast Fourier transforms<a id='math:sec:fast_fourier_tranforms'></a>

The discrete Fourier transform is a computationally expensive operation. The number of complex multiplications to calculate the set of $N$ datapoints if one performs a diescrete Fourier transform of $N$ data points is $N^2$. Since the number of data points to be Fourier-transformed can become very high, one usually tries to make use of numerical tricks to reduce the number of calculations. 

It is not difficult to identify potential ways to safe computing time. Looking at the definition of the [discrete Fourier transform &#10549;](#math:eq:8_001)<!--\ref{math:eq:8_001}-->, one can see that -- under certain circumstances -- the same summands occur multiple times. E.g. for $N\,=\,8$, the product $y_2\,e^{-2{\pi}i\frac{4}{8}}=y_2e^{-2{\pi}i\frac{12}{8}}$ has to be calculated for $\mathscr{F}_{\rm D}\{y\}_2$ but also for $\mathscr{F}_{\rm D}\{y\}_6$. Would we have to calculate the Fourier transform by hand, we would not calculate this summand twice.

A lot of effort has gone into implementing algorithms that make use of this and other identities. Cooley und Tukey published an algorithm ("The Fast Fourier Transform" or "FFT") in 1965, which, for a length of $N\,=\,2^n\quad n\in\mathbb{N}$ of the discrete Fourier transform, enables to reduce the number of complex multiplications to $2N{\log}_2N$, and made it possible for the first time to perform a discrete Fourier transform of larger sequences of numbers.

Today's implementations of discrete Fourier transforms make use of the FFT and other algorithms hiding some provisions that the user had to make in the past (e.g. regrid the data series on a grid with $2^n$ cells), such that a detailed treatment of the algorithms is not required. However, from time to time one has to deal with the details and the reader should be prepared to learn a bit more about implementations of fast Fourier transforms. Usually, however, this is well described in the manuals.

***

* Next: [2.9 Sampling Theory](2_9_sampling_theory.ipynb)